# Model Context Protocol (MCP) Integratiion

This notebook demonstrates how to combine MCP Servers with the Langchain + Snowflake integration to leverage Snowflake Cortex models as the orchestrating model in MCP.

## What You'll Learn

1. **MCP Tools** - Unpacking MCP server tools as Langchain tools
2. **MCP Server Connection** - Connect Langchain chat with an MCP servers 

## Prerequisites

- Completed `getting_started.ipynb` and `snowflake_workflows.ipynb`
- Understanding of LangChain basics (chat models, tools, chains)
- Understanding of Model Context Protocol

## Why These Patterns Matter

The Langchain - Snowflake integration is the first open-source framework that combines Model Context Protocol with Snowflake Cortex LLMs as the orchestrating models. This combination allows an end-to-end MCP ecosystem to exist entirely within the Snowflake security perimeter.


## Installation

Install additional package:


In [ ]:
# %pip install --quiet -U langchain-mcp-adapters

[langchain-mcp-adapters](https://github.com/langchain-ai/langchain-mcp-adapters) provides a lightweight wrapper that makes MCP Server tools compatible with LangChain binded tools. 

## Setup

In [2]:
## Setup
from langchain_snowflake import ChatSnowflake, create_session_from_env
from langchain_mcp_adapters.tools import load_mcp_tools
from mcp import ClientSession


# Initialize session and LLM - consistent with other notebooks
session = create_session_from_env()
llm = ChatSnowflake(
    session=session, model="claude-4-sonnet", temperature=0.1, max_tokens=2000
)

print("Session and LLM initialized successfully")

Session and LLM initialized successfully


## MCP Server
We will use the [open-source MCP Server for Snowflake](https://github.com/Snowflake-Labs/mcp/tree/main) as an example. The MCP Server supports `stdio`, `streamable-http`, and `sse` transports. 

## 1. Asynchronous Tool Calling with STDIO MCP Server

In [ ]:
from mcp import StdioServerParameters
from mcp.client.stdio import stdio_client

server_params = StdioServerParameters(
    command="uvx",
    args=[
        "snowflake-labs-mcp", 
        "--service-config-file", 
        "/Users/jsummer/.snowflake/mcp_config.yaml", # Change to configuration file path
        "--connection-name", 
        "default", 
        "--transport",
        "stdio"
        ],
)

prompt = "create a transient database named my_langchain_db with replacement"

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as client_session:
        
        # Initialize the connection
        await client_session.initialize()

        # Get tools
        tools = await load_mcp_tools(client_session)
        agent = llm.bind_tools(tools)

        # Use sync invoke for the LLM, but handle async tools manually
        response = agent.invoke(prompt)
        print("Initial response:", response)
        
        # If tools were called, execute them manually
        if hasattr(response, 'tool_calls') and response.tool_calls:
            print(f"Tools called: {len(response.tool_calls)}")
            for tool_call in response.tool_calls:
                tool_name = tool_call['name']
                tool_args = tool_call['args']
                print(f"Executing tool: {tool_name} with args: {tool_args}")

                result = await client_session.call_tool(tool_name, tool_args)
                for content in result.content:
                    if content.type == 'text':
                        print(content.text)

        else:
            print(response.content)

Initial response: content='I\'ll create a transient database named "my_langchain_db" with replacement mode for you.' additional_kwargs={} response_metadata={'model': 'claude-4-sonnet', 'model_name': 'claude-4-sonnet', 'finish_reason': 'tool_calls'} id='run--1cb7b413-9bbb-44ef-8bb3-e1bafeb30b6f-0' tool_calls=[{'name': 'create_object', 'args': {'object_type': 'database', 'mode': 'replace', 'target_object': {'name': 'my_langchain_db', 'kind': 'TRANSIENT'}}, 'id': 'toolu_bdrk_017mBeWEVvyHFNC2UeUdZYhM', 'type': 'tool_call'}] usage_metadata={'input_tokens': 18462, 'output_tokens': 138, 'total_tokens': 18600}
Tools called: 1
Executing tool: create_object with args: {'object_type': 'database', 'mode': 'replace', 'target_object': {'name': 'my_langchain_db', 'kind': 'TRANSIENT'}}
Created Database my_langchain_db.


## 2. Asynchronous Tool Calling with HTTP-streaming MCP Server

Start the MCP Server with streamable-http transport. Endpoint `http://0.0.0.0:9000/mcp/` surfaced...

In [ ]:
from mcp.client.streamable_http import streamablehttp_client


prompt = "create a transient database named my_langchain_db with replacement"

async with streamablehttp_client("http://0.0.0.0:9000/mcp/") as (read, write, _):
    async with ClientSession(read, write) as client_session:
        # Initialize the connection
        await client_session.initialize()

        # Get tools
        tools = await load_mcp_tools(client_session)
        agent = llm.bind_tools(tools)

        # Use sync invoke for the LLM, but handle async tools manually
        response = agent.invoke(prompt)
        print("Initial response:", response)
        
        # If tools were called, execute them manually
        if hasattr(response, 'tool_calls') and response.tool_calls:
            print(f"Tools called: {len(response.tool_calls)}")
            for tool_call in response.tool_calls:
                tool_name = tool_call['name']
                tool_args = tool_call['args']
                print(f"Executing tool: {tool_name} with args: {tool_args}")

                result = await client_session.call_tool(tool_name, tool_args)
                for content in result.content:
                    if content.type == 'text':
                        print(content.text)
                        
        else:
            print(response.content)

Initial response: content='I\'ll create a transient database named "my_langchain_db" with replacement mode for you.' additional_kwargs={} response_metadata={'model': 'claude-4-sonnet', 'model_name': 'claude-4-sonnet', 'finish_reason': 'tool_calls'} id='run--cdc66d35-423e-44d7-84b8-1931ce2e42ab-0' tool_calls=[{'name': 'create_object', 'args': {'object_type': 'database', 'mode': 'replace', 'target_object': {'name': 'my_langchain_db', 'kind': 'TRANSIENT'}}, 'id': 'toolu_bdrk_019HeD351AgVFG9BnSpgXgwF', 'type': 'tool_call'}] usage_metadata={'input_tokens': 18462, 'output_tokens': 138, 'total_tokens': 18600}
Tools called: 1
Executing tool: create_object with args: {'object_type': 'database', 'mode': 'replace', 'target_object': {'name': 'my_langchain_db', 'kind': 'TRANSIENT'}}
Created Database my_langchain_db.


## Extra Resources
- https://github.com/langchain-ai/langchain-snowflake/blob/main/README.md
- https://github.com/langchain-ai/langchain-snowflake/blob/main/libs/snowflake/README.md